# Importar librerias

In [71]:
import numpy as np

import matplotlib.pyplot as plt
import sklearn
%load_ext cudf.pandas
# pandas API is now GPU accelerated

import pandas as pd

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


# Importar datos

In [72]:
#Cargamos los datos
! pip install openpyxl
data = pd.read_excel('data/bosch_aic_datathon.xlsx')

# Hacemos copia de los datos
data_sucia = data
data_sucia.head()

,Posición,Area,Á.liq.,Grupo de personal,CODIGO POSTAL,Motivo de la RENUNCIA,Banda,Baja,ReglaPHT,Alta,Años,Meses,Antigüedad,Clave de sexo,Lugar de nacimiento,Nacionalidad,Edad del empleado,¿Cuanto tiempo tiene viviendo en Cd. Juarez?,Estado Civil,Hijos
0,POS514,LOG,S6,Exempt,32520,Vol-Otro empleo,E3,2023-01-02,ADMINIST,2018-06-21,5,55.200000,1656,Femenino,JUAREZ,Mexican,31,None,solter,0
1,POS1745,MFS,S5,Direct,32217,Vol-Otro empleo,DNO,2023-01-03,ADMINIST,2017-07-20,6,66.433333,1993,Masculino,DURANGO,Mexican,25,10+ Years,solter,0
2,POS1230,VS7,S5,Direct,32600,Vol-Otro empleo,DNO,2023-01-03,JT012324,2019-08-29,4,40.766667,1223,Masculino,JUAREZ,Mexican,23,10+ Years,solter,0
3,POS1661,MFS,S5,Direct,32575,Vol-Estudios,DO,2023-01-03,JT012324,2022-11-03,1,2.033333,61,Femenino,CIUDAD JUAREZ,Mexican,18,10+ Years,solter,0
4,POS1921,MFS,S5,Direct,32575,Vol-Cambio de domicilio,DO,2023-01-03,JT012324,2017-03-02,6,71.100000,2133,Masculino,JUAREZ,Mexican,26,10+ Years,solter,0


# Limpieza por columnas

In [73]:
# Poner todas las columnas en mayúsculas
data_sucia.columns = map(str.upper, data_sucia.columns)
data_sucia.head()

,POSICIÓN,AREA,Á.LIQ.,GRUPO DE PERSONAL,CODIGO POSTAL,MOTIVO DE LA RENUNCIA,BANDA,BAJA,REGLAPHT,ALTA,AÑOS,MESES,ANTIGÜEDAD,CLAVE DE SEXO,LUGAR DE NACIMIENTO,NACIONALIDAD,EDAD DEL EMPLEADO,¿CUANTO TIEMPO TIENE VIVIENDO EN CD. JUAREZ?,ESTADO CIVIL,HIJOS
0,POS514,LOG,S6,Exempt,32520,Vol-Otro empleo,E3,2023-01-02,ADMINIST,2018-06-21,5,55.200000,1656,Femenino,JUAREZ,Mexican,31,None,solter,0
1,POS1745,MFS,S5,Direct,32217,Vol-Otro empleo,DNO,2023-01-03,ADMINIST,2017-07-20,6,66.433333,1993,Masculino,DURANGO,Mexican,25,10+ Years,solter,0
2,POS1230,VS7,S5,Direct,32600,Vol-Otro empleo,DNO,2023-01-03,JT012324,2019-08-29,4,40.766667,1223,Masculino,JUAREZ,Mexican,23,10+ Years,solter,0
3,POS1661,MFS,S5,Direct,32575,Vol-Estudios,DO,2023-01-03,JT012324,2022-11-03,1,2.033333,61,Femenino,CIUDAD JUAREZ,Mexican,18,10+ Years,solter,0
4,POS1921,MFS,S5,Direct,32575,Vol-Cambio de domicilio,DO,2023-01-03,JT012324,2017-03-02,6,71.100000,2133,Masculino,JUAREZ,Mexican,26,10+ Years,solter,0


### Posicion

### Area

### A Liquida

In [74]:
# Convierte columna Á.liq. a categoricos
data_sucia['Á.liq.'] = data_sucia['Á.liq.'].astype('category')
# Hace plot de pie de la columna Á.liq.
data_sucia['Á.liq.'].value_counts().plot(kind='pie', autopct='%.2f', figsize=(6, 6))


KeyError: 'Á.liq.'

In [ ]:
# C

### Grupo de personal

### Codigo postal

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

geolocator = Nominatim(user_agent="my_geocoder")

# Obtener coordenadas de geolocalización de un código postal
def obtener_coordenadas_osm(codigo_postal):
    location = geolocator.geocode({"postalcode": codigo_postal})
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Funcion para calcular la distancia entre dos codigos postales
def calcular_distancia_osm(cp_empleado, cp_bosch):
    coords_empleado = obtener_coordenadas_osm(cp_empleado)
    coords_bosch = obtener_coordenadas_osm(cp_bosch)

    if coords_empleado and coords_bosch:
        distancia = geodesic(coords_empleado, coords_bosch).kilometers
        return distancia
    else:
        return None
# Insertar una columna llamada "Codigo Postal Bosch" con el valor 32557
data_sucia['Codigo Postal Bosch'] = 32557

# Calcular la distancia entre el empleado y Bosch
if 'Codigo Postal' in data_sucia.columns and 'Codigo Postal Bosch' in data_sucia.columns:
    data_sucia['distancia'] = data_sucia.apply(lambda x: calcular_distancia_osm(x['C.P.'], x['C.P. Bosch']), axis=1)
    data_sucia.head()
else:
    print("Column names 'C.P.' and/or 'C.P. Bosch' not found in the DataFrame.")
def calcular_distancia_osm(cp_empleado, cp_bosch):
    coords_empleado = obtener_coordenadas_osm(cp_empleado)
    coords_bosch = obtener_coordenadas_osm(cp_bosch)

    if coords_empleado and coords_bosch:
        distancia = geodesic(coords_empleado, coords_bosch).kilometers
        return distancia
    else:
        return None

# Calcular la distancia entre el empleado y Bosch
data_sucia['distancia'] = data_sucia.apply(lambda x: calcular_distancia_osm(x['Codigo Postal'], x['Codigo Postal Bosch']), axis=1)
data_sucia.head()


Column names 'C.P.' and/or 'C.P. Bosch' not found in the DataFrame.


KeyError: 'Codigo Postal'

### Motivo de la renuncia

### Motivo de la renuncia

### Baja

### Alta

### Regla PHT

### Años

### Meses

### Dias totales/Antigüedad

### Sexo

### Lugar de nacimiento

In [ ]:
# Limpia los datos de la columna Lugar de nacimiento a el nombre del estado de nacimiento. 
# Limpia los datos de la columna Lugar de nacimiento a el nombre del estado de nacimiento. 
# Diccionario de reemplazos
reemplazos = {
    'Chihuahua': ['AZTECA CIUDAD JUAREZ', 'CAMARGO', 'CD JUAREZ', 'CD JUAREZ CHIH', 'CD. JUAREZ'],
    'Durango': ['CD. GOMEZ PALACIO', 'GENERAL SIMON BOLIVAR', 'GOMEZ PALACIO', 'LERDO', 'DURANGO', 'PEÑON BLANCO', 'POBLADO EL LUCERO', 'SAN DIMAS', 'TLAHUALILO DE ZARAGOZA'],
    'Aguascalientes': ['AGUASCALIENTES'],
    'Sinaloa': ['AHOME', 'CULIACAN'],
    'Veracruz': ['ANGEL R CABADA', 'COATZACOALCOS', 'CORDOBA', 'CORDOVA', 'COSAMALOAPAN', 'LAS MERCEDES', 'MARTINEZ DE LA TORRE', 'MINATITLAN', 'MOLOACAN', 'RAYA CARACOL', 'SAN ANDRES TUXTLA', 'TRES VALLES', 'VERACRUZ'],
    # Agrega el resto de tus reemplazos aquí
}

# Invertir el diccionario para mapear cada ciudad a su estado correspondiente
reemplazos_invertidos = {ciudad: estado for estado, ciudades in reemplazos.items() for ciudad in ciudades}

# Realizar los reemplazos
data_sucia['Lugar de nacimiento'] = data_sucia['Lugar de nacimiento'].replace(reemplazos_invertidos)

# Ver resultados de los reemplazos usando aleatoriedad
data_sucia['Lugar de nacimiento'].sample(10)

# Diccionario para la nueva celda de reemplazos
reemplazos_nueva_celda = {
    'PUEBLA': ['GUADALUPE ANALCO', 'PUEBLA'],
    'OAXACA': ['IXHUATAN', 'OAXACA', 'PASO DEL JOBO', 'SAN FRANCISCO DEL MAR', 'SAN JUAN BAUTISTA TUXTEPE', 'SAN LUCAS OJITLAN', 'TUXTEPEC'],
    'TAMAULIPAS': ['MATAMOROS']
}

# Invertir el diccionario para mapear cada ciudad a su estado correspondiente
reemplazos_invertidos_nueva_celda = {ciudad: estado for estado, ciudades in reemplazos_nueva_celda.items() for ciudad in ciudades}

# Realizar los reemplazos
data_sucia['Lugar de nacimiento'] = data_sucia['Lugar de nacimiento'].replace(reemplazos_invertidos_nueva_celda)

data_sucia['Lugar de nacimiento'].sample(10)


1275               OAXACA
1369               OCAMPO
347         CIUDAD JUAREZ
1294            CHIHUAHUA
970              Veracruz
51                 JUAREZ
1357               JUAREZ
866                JUAREZ
798     CD. JUAREZ, CHIH.
470                JUAREZ
Name: Lugar de nacimiento, dtype: object

### Nacionalidad

### Edad del empleado

In [ ]:
# Edad del empleado
# Convertir en tipos de datos enteros
data_sucia['Edad del empleado'] = data_sucia['Edad del empleado'].astype(int)
# Cambiar nombre de columna a Edad
data_sucia = data_sucia.rename(columns={'Edad del empleado': 'Edad'})

# Ver resultados de los reemplazos usando aleatoriedad

### Tiempo viviendo en Ciudad Juarez

In [ ]:
# Categorizar "Tiempo Viviendo en Ciudad Juárez"

def categorize_time_living(time):
    if pd.isna(time) or isinstance(time, str) and 'Days' in time:
        return 'Menos de 1 Año'
    elif isinstance(time, str) and any(x in time for x in ['1-2 Years', '2-3 Years']) or (isinstance(time, int) and 1 <= time <= 3):
        return '1-3 Años'
    elif isinstance(time, str) and any(x in time for x in ['3-4 Years', '4-5 Years', '5-6 Years']) or (isinstance(time, int) and 4 <= time <= 6):
        return '4-6 Años'
    elif isinstance(time, str) and any(x in time for x in ['6-7 Years', '7-8 Years', '8-9 Years']) or (isinstance(time, int) and 7 <= time <= 9):
        return '7-9 Años'
    else:
        return '10+ Años'

# Apply the categorization
data_sucia['Tiempo Viviendo en Cd. Juarez - Categoria'] = data_sucia['¿Cuanto tiempo tiene viviendo en Cd. Juarez?'].apply(categorize_time_living)

# Display the first few rows to verify the categorization
data_sucia[['¿Cuanto tiempo tiene viviendo en Cd. Juarez?', 'Tiempo Viviendo en Cd. Juarez - Categoria']].head()

# Saber cuantos valores hay en cada categoria de la columna Tiempo Viviendo en Cd. Juarez - Categoría
data_sucia['Tiempo Viviendo en Cd. Juarez - Categoria'].value_counts()

Menos de 1 Año    788
10+ Años          508
4-6 Años           45
7-9 Años           42
1-3 Años           20
Name: Tiempo Viviendo en Cd. Juarez - Categoria, dtype: int64

### Estado civil

### Hijos - Victor

In [ ]:
# Convertir columna hijos a enteros
data_sucia['Hijos'] = data_sucia['Hijos'].astype(int)

# Crear una columna con nombre 'Tiene Hijos' que sera 'Sí' si 'Hijos' > 0 y 'No' si 'Hijos' == 0
data_sucia['Tiene Hijos'] = data['Hijos'].apply(lambda x: 'Sí' if x > 0 else 'No')

# Convertir columna Tiene Hijos booleana
data_sucia['Tiene Hijos'] = data_sucia['Tiene Hijos'].astype('category')


----
# Subida de datos en DB MySQL

In [ ]:
"""# Importar libreria para trabajar con mysql
#! pip install mysql-connector-python
import mysql.connector

# Hacer un mapeo de los datos necesarios para conectarse
config = {
    'user': 'datathonequipo5',
    'password': 'CodedSpiritDev05',
    'host': 'datathon.mysql.database.azure.com',
    'database': 'datathon'
}

# Conectarse a la base de datos
cnx = mysql.connector.connect(**config)

# Verificar la conexión
cnx.is_connected()

# Cerrar la conexión
cnx.close()"""

"# Importar libreria para trabajar con mysql\n#! pip install mysql-connector-python\nimport mysql.connector\n\n# Hacer un mapeo de los datos necesarios para conectarse\nconfig = {\n    'user': 'datathonequipo5',\n    'password': 'CodedSpiritDev05',\n    'host': 'datathon.mysql.database.azure.com',\n    'database': 'datathon'\n}\n\n# Conectarse a la base de datos\ncnx = mysql.connector.connect(**config)\n\n# Verificar la conexión\ncnx.is_connected()\n\n# Cerrar la conexión\ncnx.close()"